import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12,6)

print("Libraries imported successfully!")
print(f"DuckDB version: {duckdb.__version__}")
print(f"Pandas version: {pd.__version__}")


In [34]:
!pip install duckdb 

import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12,6)

print("Libraries imported successfully!")
print(f"DuckDB version: {duckdb.__version__}")
print(f"Pandas version: {pd.__version__}")

Python(33462) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Libraries imported successfully!
DuckDB version: 1.4.2
Pandas version: 2.3.3


## 1. Setup and Library Imports

In [35]:
%run ../src/read_s3_data.py

Reading SCMS_Delivery_History_Dataset.csv from s3://ml-supplychain-project ...
Data read successfully!
   ID Project Code            PQ # PO / SO # ASN/DN #        Country  \
0   1   100-CI-T01  Pre-PQ Process    SCMS-4    ASN-8  Côte d'Ivoire   
1   3   108-VN-T01  Pre-PQ Process   SCMS-13   ASN-85        Vietnam   
2   4   100-CI-T01  Pre-PQ Process   SCMS-20   ASN-14  Côte d'Ivoire   
3  15   108-VN-T01  Pre-PQ Process   SCMS-78   ASN-50        Vietnam   
4  16   108-VN-T01  Pre-PQ Process   SCMS-81   ASN-55        Vietnam   

  Managed By  Fulfill Via Vendor INCO Term Shipment Mode  ...  \
0   PMO - US  Direct Drop              EXW           Air  ...   
1   PMO - US  Direct Drop              EXW           Air  ...   
2   PMO - US  Direct Drop              FCA           Air  ...   
3   PMO - US  Direct Drop              EXW           Air  ...   
4   PMO - US  Direct Drop              EXW           Air  ...   

  Unit of Measure (Per Pack) Line Item Quantity Line Item Value Pack Pric

In [36]:
# Parse date columns
date_columns = [
    'PQ First Sent to Client Date',
    'PO Sent to Vendor Date',
    'Scheduled Delivery Date',
    'Delivered to Client Date',
    'Delivery Recorded Date'
]

for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Create target variable
df['delay_days'] = (df['Delivered to Client Date'] - df['Scheduled Delivery Date']).dt.days
df['is_late'] = (df['delay_days'] > 0).astype(int)


print("Date columns parsed and target variable created!")

Date columns parsed and target variable created!


## 2. Load Data from S3

In [37]:
# Load data from S3 using the existing script
%run ../src/read_s3_data.py

print(f"\nData loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")

Reading SCMS_Delivery_History_Dataset.csv from s3://ml-supplychain-project ...
Data read successfully!
   ID Project Code            PQ # PO / SO # ASN/DN #        Country  \
0   1   100-CI-T01  Pre-PQ Process    SCMS-4    ASN-8  Côte d'Ivoire   
1   3   108-VN-T01  Pre-PQ Process   SCMS-13   ASN-85        Vietnam   
2   4   100-CI-T01  Pre-PQ Process   SCMS-20   ASN-14  Côte d'Ivoire   
3  15   108-VN-T01  Pre-PQ Process   SCMS-78   ASN-50        Vietnam   
4  16   108-VN-T01  Pre-PQ Process   SCMS-81   ASN-55        Vietnam   

  Managed By  Fulfill Via Vendor INCO Term Shipment Mode  ...  \
0   PMO - US  Direct Drop              EXW           Air  ...   
1   PMO - US  Direct Drop              EXW           Air  ...   
2   PMO - US  Direct Drop              FCA           Air  ...   
3   PMO - US  Direct Drop              EXW           Air  ...   
4   PMO - US  Direct Drop              EXW           Air  ...   

  Unit of Measure (Per Pack) Line Item Quantity Line Item Value Pack Pric

In [38]:
# Parse date columns
date_columns = [
    'PQ First Sent to Client Date',
    'PO Sent to Vendor Date',
    'Scheduled Delivery Date',
    'Delivered to Client Date',
    'Delivery Recorded Date'
]

for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Create target variable
df['delay_days'] = (df['Delivered to Client Date'] - df['Scheduled Delivery Date']).dt.days
df['is_late'] = (df['delay_days'] > 0).astype(int)

print("Date columns parsed and target variable created!")

Date columns parsed and target variable created!


# Query 1: Delay rate by country
query = """
SELECT 
    Country,
    COUNT(*) as total_shipments,
    SUM(CASE WHEN is_late = 1 THEN 1 ELSE 0 END) as late_deliveries,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate,
    ROUND(AVG(delay_days), 2) as avg_delay_days,
    ROUND(AVG("Freight Cost (USD)"), 2) as avg_freight_cost
FROM shipments
WHERE Country IS NOT NULL
GROUP BY Country
HAVING COUNT(*) >= 10
ORDER BY delay_rate DESC
LIMIT 15
"""

country_delay_rates = conn.execute(query).df()
print("Delay Rate by Country (Top 15):")
print("="*80)
print(country_delay_rates)

# Visualze delay rates (minor typo)
plt.figure(figsize=(12, 6))
plt.barh(country_delay_rates['Country'], country_delay_rates['delay_rate'])
plt.xlabel('Delay Rate')
plt.title('Top 15 Countries by Delivery Delay Rate (SQL Query)')
plt.tight_layout()
plt.show()

In [39]:
# Create database directory if it doesn't exist
db_path = Path('../data/processed')
db_path.mkdir(parents=True, exist_ok=True)

# Initialize DuckDB connection
db_file = '../data/processed/supply_chain.db'
conn = duckdb.connect(db_file)

print(f"DuckDB database initialized: {db_file}")
print(f"Connection: {conn}")

DuckDB database initialized: ../data/processed/supply_chain.db
Connection: <_duckdb.DuckDBPyConnection object at 0x11fa6a270>


In [40]:
# Create table from pandas DataFrame
conn.execute("DROP TABLE IF EXISTS shipments")
conn.execute("CREATE TABLE shipments AS SELECT * FROM df")

# Verify table creation
row_count = conn.execute("SELECT COUNT(*) FROM shipments").fetchone()[0]
print(f"Table 'shipments' created with {row_count:,} rows")

# Show table schema
print("\nTable Schema:")
schema = conn.execute("DESCRIBE shipments").df()
print(schema)

Table 'shipments' created with 10,324 rows

Table Schema:
                     column_name   column_type null   key default extra
0                             ID        BIGINT  YES  None    None  None
1                   Project Code       VARCHAR  YES  None    None  None
2                           PQ #       VARCHAR  YES  None    None  None
3                      PO / SO #       VARCHAR  YES  None    None  None
4                       ASN/DN #       VARCHAR  YES  None    None  None
5                        Country       VARCHAR  YES  None    None  None
6                     Managed By       VARCHAR  YES  None    None  None
7                    Fulfill Via       VARCHAR  YES  None    None  None
8               Vendor INCO Term       VARCHAR  YES  None    None  None
9                  Shipment Mode       VARCHAR  YES  None    None  None
10  PQ First Sent to Client Date  TIMESTAMP_NS  YES  None    None  None
11        PO Sent to Vendor Date  TIMESTAMP_NS  YES  None    None  None
12    

## 4. Load Data into DuckDB

In [41]:
# Create a table with derived date features
query = """
CREATE OR REPLACE TABLE shipments_with_features AS
SELECT 
    *,
    -- Date features from Scheduled Delivery Date
    EXTRACT(YEAR FROM "Scheduled Delivery Date") as scheduled_year,
    EXTRACT(MONTH FROM "Scheduled Delivery Date") as scheduled_month,
    EXTRACT(QUARTER FROM "Scheduled Delivery Date") as scheduled_quarter,
    EXTRACT(DOW FROM "Scheduled Delivery Date") as scheduled_day_of_week,
    EXTRACT(DAY FROM "Scheduled Delivery Date") as scheduled_day_of_month,
    
    -- Lead time calculation
    DATEDIFF('day', "PO Sent to Vendor Date", "Scheduled Delivery Date") as lead_time_days,
    
    -- Financial features (cast VARCHAR to DOUBLE for arithmetic)
    CASE 
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) > 0 
        THEN ROUND(CAST("Freight Cost (USD)" AS DOUBLE) / CAST("Weight (Kilograms)" AS DOUBLE), 2)
        ELSE NULL 
    END as cost_per_kg,
    
    CASE 
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) > 0 
        THEN ROUND("Line Item Value" / CAST("Weight (Kilograms)" AS DOUBLE), 2)
        ELSE NULL 
    END as value_per_kg,
    
    -- Categorical features
    CASE 
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) < 10 THEN 'Light'
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) < 100 THEN 'Medium'
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) < 1000 THEN 'Heavy'
        ELSE 'Very Heavy'
    END as weight_category,
    
    CASE 
        WHEN "Line Item Value" < 1000 THEN 'Low'
        WHEN "Line Item Value" < 10000 THEN 'Medium'
        WHEN "Line Item Value" < 50000 THEN 'High'
        ELSE 'Very High'
    END as value_category,
    
    -- End of month/quarter indicaters (minor typo in comment)
    CASE WHEN EXTRACT(DAY FROM "Scheduled Delivery Date") > 25 THEN 1 ELSE 0 END as is_month_end,
    CASE WHEN EXTRACT(MONTH FROM "Scheduled Delivery Date") IN (3, 6, 9, 12) 
              AND EXTRACT(DAY FROM "Scheduled Delivery Date") > 25 THEN 1 ELSE 0 END as is_quarter_end
    
FROM shipments
"""

conn.execute(query)
print("Feature engineering table created: shipments_with_features")

# Verify new features
sample = conn.execute("SELECT * FROM shipments_with_features LIMIT 5").df()
print("\nSample with new features:")
print(sample[['Country', 'scheduled_month', 'lead_time_days', 'cost_per_kg', 'weight_category', 'is_late']].head())

ConversionException: Conversion Error: Could not convert string 'See ASN-93 (ID#:1281)' to DOUBLE when casting from source column Weight (Kilograms)

LINE 17:         WHEN CAST("Weight (Kilograms)" AS DOUBLE) > 0 
                      ^

# Create vendor metrics table
query = """
CREATE OR REPLACE TABLE vendor_metrics AS
SELECT 
    Vendor,
    COUNT(*) as total_shipments,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as vendor_delay_rate,
    ROUND(AVG(delay_days), 2) as vendor_avg_delay_days,
    ROUND(STDDEV(delay_days), 2) as vendor_delay_std,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as vendor_avg_freight_cost,
    ROUND(AVG(CAST("Weight (Kilograms)" AS DOUBLE)), 2) as vendor_avg_weight
FROM shipments
WHERE Vendor IS NOT NULL
GROUP BY Vendor
"""

conn.execute(query)
print("Vendor metrics table created")

# Preview vendor metrics
vendor_metrics_sample = conn.execute("""
    SELECT * FROM vendor_metrics 
    WHERE total_shipments >=20 
    ORDER BY vendor_delay_rate 
    LIMIT 10
""").df()
print("\nTop 10 Vendors by Reliability (min 20 shipments):")
print(vendor_metrics_sample)

In [ ]:
# Check for duplicate records
query = """
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT ID) as unique_ids,
    COUNT(*) - COUNT(DISTINCT ID) as duplicate_ids
FROM shipments
"""

duplicate_check = conn.execute(query).df()
print("Duplicate Check:")
print("="*50)
print(duplicate_check)

In [ ]:
# Missing value analysis using SQL
query = """
SELECT 
    COUNT(*) as total_rows,
    COUNT(*) - COUNT("Scheduled Delivery Date") as missing_scheduled_date,
    COUNT(*) - COUNT("Delivered to Client Date") as missing_delivered_date,
    COUNT(*) - COUNT("Freight Cost (USD)") as missing_freight_cost,
    COUNT(*) - COUNT("Weight (Kilograms)") as missing_weight,
    COUNT(*) - COUNT("Line Item Insurance (USD)") as missing_insurance
FROM shipments
"""

missing_values = conn.execute(query).df()
print("\nMissing Values Analysis:")
print("="*50)
print(missing_values)

# Export ML-ready data to pandas DataFrame
ml_data = conn.execute("SELECT * FROM ml_ready_data").df()

print(f"ML-ready data shape: {ml_data.shape}")
print(f"\nColumns in ML-ready data: {len(ml_data.columns)}")

# Save to CSV
output_path = '../data/interim/02_sql_processed_data.csv'
ml_data.to_csv(output_path, index=False)


print(f"\nData saved to: {output_path}")

In [ ]:
# Query 1: Delay rate by country
query = """
SELECT 
    Country,
    COUNT(*) as total_shipments,
    SUM(CASE WHEN is_late = 1 THEN 1 ELSE 0 END) as late_deliveries,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate,
    ROUND(AVG(delay_days), 2) as avg_delay_days,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as avg_freight_cost
FROM shipments
WHERE Country IS NOT NULL
GROUP BY Country
HAVING COUNT(*) >= 10
ORDER BY delay_rate DESC
LIMIT 15
"""

country_delay_rates = conn.execute(query).df()
print("Delay Rate by Country (Top 15):")
print("="*80)
print(country_delay_rates)

# Visualize
plt.figure(figsize=(12, 6))
plt.barh(country_delay_rates['Country'], country_delay_rates['delay_rate'])
plt.xlabel('Delay Rate')
plt.title('Top 15 Countries by Delivery Delay Rate (SQL Query)')
plt.tight_layout()
plt.show()

In [ ]:
# Query 2: Shipment mode performance analysis
query = """
SELECT 
    "Shipment Mode",
    COUNT(*) as total_shipments,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate,
    ROUND(AVG(delay_days), 2) as avg_delay_days,
    ROUND(MIN(delay_days), 2) as min_delay_days,
    ROUND(MAX(delay_days), 2) as max_delay_days,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as avg_freight_cost,
    ROUND(AVG(CAST("Weight (Kilograms)" AS DOUBLE)), 2) as avg_weight_kg
FROM shipments
WHERE "Shipment Mode" IS NOT NULL
GROUP BY "Shipment Mode"
ORDER BY delay_rate DESC
"""

shipment_mode_analysis = conn.execute(query).df()
print("\nShipment Mode Performance Analysis:")
print("="*80)
print(shipment_mode_analysis)

In [ ]:
# Query 3: Vendor reliability ranking
query = """
SELECT 
    Vendor,
    COUNT(*) as total_shipments,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate,
    ROUND(AVG(delay_days), 2) as avg_delay_days,
    ROUND(STDDEV(delay_days), 2) as delay_variability
FROM shipments
WHERE Vendor IS NOT NULL
GROUP BY Vendor
HAVING COUNT(*) >= 20
ORDER BY delay_rate ASC
LIMIT 10
"""

top_vendors = conn.execute(query).df()
print("\nTop 10 Most Reliable Vendors (Lowest Delay Rate, min 20 shipments):")
print("="*80)
print(top_vendors)

## 7. Advanced SQL: Window Functions

In [ ]:
# Query 4: Rolling vendor performance (30-day rolling delay rate)
query = """
WITH vendor_daily AS (
    SELECT 
        Vendor,
        "Scheduled Delivery Date" as schedule_date,
        is_late,
        delay_days
    FROM shipments
    WHERE Vendor IS NOT NULL 
      AND "Scheduled Delivery Date" IS NOT NULL
)
SELECT 
    Vendor,
    schedule_date,
    is_late,
    delay_days,
    AVG(is_late) OVER (
        PARTITION BY Vendor 
        ORDER BY schedule_date 
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) as rolling_30_delay_rate,
    COUNT(*) OVER (
        PARTITION BY Vendor 
        ORDER BY schedule_date 
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) as rolling_window_size
FROM vendor_daily
ORDER BY Vendor, schedule_date
LIMIT 100
"""

rolling_vendor_performance = conn.execute(query).df()
print("Rolling Vendor Performance (30-shipment window) - Sample:")
print("="*80)
print(rolling_vendor_performance.head(20))

In [ ]:
# Query 5: Rank vendors by performance within each country
query = """
WITH vendor_country_performance AS (
    SELECT 
        Country,
        Vendor,
        COUNT(*) as shipments,
        ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate,
        ROUND(AVG(delay_days), 2) as avg_delay_days
    FROM shipments
    WHERE Country IS NOT NULL AND Vendor IS NOT NULL
    GROUP BY Country, Vendor
    HAVING COUNT(*) >= 5
)
SELECT 
    Country,
    Vendor,
    shipments,
    delay_rate,
    avg_delay_days,
    RANK() OVER (PARTITION BY Country ORDER BY delay_rate ASC) as reliability_rank
FROM vendor_country_performance
QUALIFY reliability_rank <= 3
ORDER BY Country, reliability_rank
"""

vendor_country_ranking = conn.execute(query).df()
print("\nTop 3 Most Reliable Vendors per Country:")
print("="*80)
print(vendor_country_ranking.head(30))

## 8. Complex SQL: CTEs and Joins

In [ ]:
# Query 6: Route analysis (Vendor-Country-Mode combinations)
query = """
WITH route_performance AS (
    SELECT 
        Vendor,
        Country,
        "Shipment Mode" as shipment_mode,
        COUNT(*) as total_shipments,
        AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END) as delay_rate,
        AVG(delay_days) as avg_delay_days,
        AVG(CAST("Freight Cost (USD)" AS DOUBLE)) as avg_freight_cost
    FROM shipments
    WHERE Vendor IS NOT NULL 
      AND Country IS NOT NULL 
      AND "Shipment Mode" IS NOT NULL
    GROUP BY Vendor, Country, "Shipment Mode"
    HAVING COUNT(*) >= 5
),
route_stats AS (
    SELECT 
        *,
        ROUND(delay_rate, 4) as rounded_delay_rate,
        ROUND(avg_delay_days, 2) as rounded_avg_delay,
        ROUND(avg_freight_cost, 2) as rounded_avg_freight
    FROM route_performance
)
SELECT 
    Vendor,
    Country,
    shipment_mode,
    total_shipments,
    rounded_delay_rate as delay_rate,
    rounded_avg_delay as avg_delay_days,
    rounded_avg_freight as avg_freight_cost,
    CASE 
        WHEN rounded_delay_rate < 0.1 THEN 'Excellent'
        WHEN rounded_delay_rate < 0.25 THEN 'Good'
        WHEN rounded_delay_rate < 0.5 THEN 'Fair'
        ELSE 'Poor'
    END as performance_category
FROM route_stats
ORDER BY delay_rate DESC
LIMIT 20
"""

route_analysis = conn.execute(query).df()
print("Route Analysis (Vendor-Country-Mode combinations) - Worst 20:")
print("="*100)
print(route_analysis)

## 9. Feature Engineering with SQL

In [ ]:
# Create a table with derived date features
query = """
CREATE OR REPLACE TABLE shipments_with_features AS
SELECT 
    *,
    -- Date features from Scheduled Delivery Date
    EXTRACT(YEAR FROM "Scheduled Delivery Date") as scheduled_year,
    EXTRACT(MONTH FROM "Scheduled Delivery Date") as scheduled_month,
    EXTRACT(QUARTER FROM "Scheduled Delivery Date") as scheduled_quarter,
    EXTRACT(DOW FROM "Scheduled Delivery Date") as scheduled_day_of_week,
    EXTRACT(DAY FROM "Scheduled Delivery Date") as scheduled_day_of_month,
    
    -- Lead time calculation
    DATEDIFF('day', "PO Sent to Vendor Date", "Scheduled Delivery Date") as lead_time_days,
    
    -- Financial features (cast VARCHAR to DOUBLE for arithmetic)
    CASE 
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) > 0 
        THEN ROUND(CAST("Freight Cost (USD)" AS DOUBLE) / CAST("Weight (Kilograms)" AS DOUBLE), 2)
        ELSE NULL 
    END as cost_per_kg,
    
    CASE 
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) > 0 
        THEN ROUND("Line Item Value" / CAST("Weight (Kilograms)" AS DOUBLE), 2)
        ELSE NULL 
    END as value_per_kg,
    
    -- Categorical features
    CASE 
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) < 10 THEN 'Light'
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) < 100 THEN 'Medium'
        WHEN CAST("Weight (Kilograms)" AS DOUBLE) < 1000 THEN 'Heavy'
        ELSE 'Very Heavy'
    END as weight_category,
    
    CASE 
        WHEN "Line Item Value" < 1000 THEN 'Low'
        WHEN "Line Item Value" < 10000 THEN 'Medium'
        WHEN "Line Item Value" < 50000 THEN 'High'
        ELSE 'Very High'
    END as value_category,
    
    -- End of month/quarter indicators
    CASE WHEN EXTRACT(DAY FROM "Scheduled Delivery Date") > 25 THEN 1 ELSE 0 END as is_month_end,
    CASE WHEN EXTRACT(MONTH FROM "Scheduled Delivery Date") IN (3, 6, 9, 12) 
              AND EXTRACT(DAY FROM "Scheduled Delivery Date") > 25 THEN 1 ELSE 0 END as is_quarter_end
    
FROM shipments
"""

conn.execute(query)
print("Feature engineering table created: shipments_with_features")

# Verify new features
sample = conn.execute("SELECT * FROM shipments_with_features LIMIT 5").df()
print("\nSample with new features:")
print(sample[['Country', 'scheduled_month', 'lead_time_days', 'cost_per_kg', 'weight_category', 'is_late']].head())

## 10. Create Vendor and Country Metrics Tables

In [ ]:
# Create vendor metrics table
query = """
CREATE OR REPLACE TABLE vendor_metrics AS
SELECT 
    Vendor,
    COUNT(*) as total_shipments,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as vendor_delay_rate,
    ROUND(AVG(delay_days), 2) as vendor_avg_delay_days,
    ROUND(STDDEV(delay_days), 2) as vendor_delay_std,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as vendor_avg_freight_cost,
    ROUND(AVG(CAST("Weight (Kilograms)" AS DOUBLE)), 2) as vendor_avg_weight
FROM shipments
WHERE Vendor IS NOT NULL
GROUP BY Vendor
"""

conn.execute(query)
print("Vendor metrics table created")

# Preview vendor metrics
vendor_metrics_sample = conn.execute("""
    SELECT * FROM vendor_metrics 
    WHERE total_shipments >= 20 
    ORDER BY vendor_delay_rate 
    LIMIT 10
""").df()
print("\nTop 10 Vendors by Reliability (min 20 shipments):")
print(vendor_metrics_sample)

In [ ]:
# Create country metrics table
query = """
CREATE OR REPLACE TABLE country_metrics AS
SELECT 
    Country,
    COUNT(*) as total_shipments,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as country_delay_rate,
    ROUND(AVG(delay_days), 2) as country_avg_delay_days,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as country_avg_freight_cost,
    COUNT(DISTINCT Vendor) as num_vendors,
    COUNT(DISTINCT "Shipment Mode") as num_shipment_modes
FROM shipments
WHERE Country IS NOT NULL
GROUP BY Country
"""

conn.execute(query)
print("\nCountry metrics table created")

# Preview country metrics
country_metrics_sample = conn.execute("""
    SELECT * FROM country_metrics 
    ORDER BY country_delay_rate DESC 
    LIMIT 10
""").df()
print("\nTop 10 Countries by Delay Rate:")
print(country_metrics_sample)

## 11. Create Combined View for ML Pipeline

In [ ]:
# Create a comprehensive view joining shipments with vendor and country metrics
query = """
CREATE OR REPLACE VIEW ml_ready_data AS
SELECT 
    s.*,
    vm.vendor_delay_rate,
    vm.vendor_avg_delay_days,
    vm.vendor_delay_std,
    cm.country_delay_rate,
    cm.country_avg_delay_days,
    cm.num_vendors as country_num_vendors,
    cm.num_shipment_modes as country_num_modes
FROM shipments_with_features s
LEFT JOIN vendor_metrics vm ON s.Vendor = vm.Vendor
LEFT JOIN country_metrics cm ON s.Country = cm.Country
"""

conn.execute(query)
print("ML-ready view created: ml_ready_data")

# Check the view
view_info = conn.execute("""
    SELECT COUNT(*) as row_count, 
           COUNT(*) FILTER (WHERE is_late = 1) as late_count,
           ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as overall_delay_rate
    FROM ml_ready_data
""").df()
print("\nML-Ready Data Summary:")
print(view_info)

## 12. Business Intelligence Queries

In [ ]:
# Query 7: Monthly trend analysis
query = """
SELECT 
    scheduled_year,
    scheduled_month,
    COUNT(*) as shipments,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate,
    ROUND(AVG(delay_days), 2) as avg_delay_days,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as avg_freight_cost
FROM shipments_with_features
WHERE scheduled_year IS NOT NULL AND scheduled_month IS NOT NULL
GROUP BY scheduled_year, scheduled_month
ORDER BY scheduled_year, scheduled_month
"""

monthly_trends = conn.execute(query).df()
print("Monthly Trends:")
print("="*80)
print(monthly_trends)

# Visualize monthly delay rate trend
if len(monthly_trends) > 0:
    monthly_trends['year_month'] = monthly_trends['scheduled_year'].astype(str) + '-' + \
                                    monthly_trends['scheduled_month'].astype(str).str.zfill(2)
    
    plt.figure(figsize=(14, 5))
    plt.plot(monthly_trends['year_month'], monthly_trends['delay_rate'], marker='o')
    plt.title('Delivery Delay Rate Over Time (SQL Query)')
    plt.xlabel('Month')
    plt.ylabel('Delay Rate')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Query 8: Cost analysis by weight category and shipment mode
query = """
SELECT 
    weight_category,
    "Shipment Mode" as shipment_mode,
    COUNT(*) as shipments,
    ROUND(AVG(CAST("Freight Cost (USD)" AS DOUBLE)), 2) as avg_freight_cost,
    ROUND(AVG(cost_per_kg), 2) as avg_cost_per_kg,
    ROUND(AVG(CASE WHEN is_late = 1 THEN 1.0 ELSE 0.0 END), 4) as delay_rate
FROM shipments_with_features
WHERE weight_category IS NOT NULL AND "Shipment Mode" IS NOT NULL
GROUP BY weight_category, "Shipment Mode"
ORDER BY weight_category, avg_freight_cost DESC
"""

cost_analysis = conn.execute(query).df()
print("\nCost Analysis by Weight Category and Shipment Mode:")
print("="*80)
print(cost_analysis)

## 13. Export Processed Data

In [ ]:
# Export ML-ready data to pandas DataFrame
ml_data = conn.execute("SELECT * FROM ml_ready_data").df()

print(f"ML-ready data shape: {ml_data.shape}")
print(f"\nColumns in ML-ready data: {len(ml_data.columns)}")

# Save to CSV
output_path = '../data/interim/02_sql_processed_data.csv'
ml_data.to_csv(output_path, index=False)
print(f"\nData saved to: {output_path}")

In [ ]:
# Show sample of processed data with new features
feature_cols = [
    'Country', 'Vendor', 'Shipment Mode',
    'is_late', 'delay_days',
    'scheduled_month', 'lead_time_days', 'cost_per_kg',
    'weight_category', 'vendor_delay_rate', 'country_delay_rate'
]

available_cols = [col for col in feature_cols if col in ml_data.columns]

print("\nSample of processed data with SQL-derived features:")
print("="*80)
print(ml_data[available_cols].head(10))

## 14. Database Summary & Available Tables

In [ ]:
# List all tables and views
tables = conn.execute("""
    SELECT table_name, table_type 
    FROM information_schema.tables 
    WHERE table_schema = 'main'
""").df()

print("Database Objects Created:")
print("="*50)
print(tables)

# Show row counts
print("\nRow Counts:")
print("="*50)
for table_name in tables['table_name']:
    count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"{table_name}: {count:,} rows")

## 15. Summary & Key SQL Demonstrations

In [ ]:
print("="*80)
print("SUMMARY - SQL DATA PROCESSING")
print("="*80)

print("\n✅ SQL Skills Demonstrated:")
print("   1. Database setup and table creation (DuckDB)")
print("   2. Aggregation queries (GROUP BY, HAVING)")
print("   3. Window functions (rolling calculations, RANK)")
print("   4. CTEs (WITH clauses) and subqueries")
print("   5. JOINs (LEFT JOIN for metrics enrichment)")
print("   6. Date functions (EXTRACT, DATEDIFF)")
print("   7. CASE statements for feature engineering")
print("   8. Views creation for ML pipeline")

print("\n📊 Database Objects Created:")
print("   - shipments (main table)")
print("   - shipments_with_features (with derived features)")
print("   - vendor_metrics (aggregated vendor stats)")
print("   - country_metrics (aggregated country stats)")
print("   - ml_ready_data (view combining all features)")

print("\n🎯 Features Created via SQL:")
print("   - Date features (month, quarter, day of week)")
print("   - Lead time calculations")
print("   - Cost per kg, value per kg")
print("   - Weight and value categories")
print("   - Vendor historical performance metrics")
print("   - Country historical metrics")
print("   - End of month/quarter indicators")

print("\n📁 Data Exported:")
print(f"   - {output_path}")
print(f"   - Shape: {ml_data.shape}")

print("\n➡️ Next Steps:")
print("   - Notebook 3: Feature Engineering (additional features)")
print("   - Notebook 4: Baseline ML Models")
print("="*80)

In [ ]:
# Close database connection
conn.close()
print("\nDuckDB connection closed.")